In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the datasets
detect_data = pd.read_csv('detect_dataset-for-3phase-machine.csv')
class_data = pd.read_csv('classData-for-3phase-machine.csv')

# Remove empty columns from detect dataset
detect_data_cleaned = detect_data.drop(columns=['Unnamed: 7', 'Unnamed: 8'])

# Normalize the data
scaler = MinMaxScaler()
detect_data_normalized = scaler.fit_transform(detect_data_cleaned)
class_data_normalized = scaler.fit_transform(class_data)

# Split the detect dataset into features and labels
X_detect = detect_data_normalized[:, 1:]  # Features
y_detect = detect_data_normalized[:, 0]   # Labels

# Split the class dataset into features and labels
X_class = class_data_normalized[:, 4:]  # Features
y_class = class_data_normalized[:, :4]  # Labels

# Further split the data into training and testing sets
X_train_detect, X_test_detect, y_train_detect, y_test_detect = train_test_split(X_detect, y_detect, test_size=0.2, random_state=42)
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)


In [7]:
pd.DataFrame(class_data_normalized)
pd.DataFrame(detect_data_normalized)

,0,1,2,3,4,5,6
0,0.0,0.403028,0.508126,0.585337,0.548701,0.000000,0.997697
1,0.0,0.430292,0.506422,0.560017,0.587308,0.024312,0.932806
2,0.0,0.448420,0.505107,0.543364,0.619020,0.042432,0.882066
3,0.0,0.454217,0.504316,0.538410,0.631410,0.044794,0.867087
4,0.0,0.463271,0.503306,0.530447,0.651058,0.053129,0.838493
...,...,...,...,...,...,...,...
11996,0.0,0.461942,0.524456,0.508940,0.581149,0.083788,0.876617
11997,0.0,0.462161,0.523902,0.509278,0.588753,0.081329,0.871547
11998,0.0,0.462389,0.523347,0.509609,0.596333,0.078980,0.866384
11999,0.0,0.462624,0.522792,0.509933,0.603888,0.076741,0.861131


In [ ]:
def balance_data(X, y):
    # Combine the data
    data = np.hstack((X, y.reshape(-1, 1)))
    # Separate the classes
    classes = np.unique(y)
    max_size = max(Counter(y).values())
    resampled_data = []

    for cls in classes:
        cls_data = data[data[:, -1] == cls]
        resampled_cls_data = resample(cls_data, replace=True, n_samples=max_size, random_state=42)
        resampled_data.append(resampled_cls_data)

    resampled_data = np.vstack(resampled_data)
    np.random.shuffle(resampled_data)

    X_resampled = resampled_data[:, :-1]
    y_resampled = resampled_data[:, -1]

    return X_resampled, y_resampled

# Balance the detect dataset
X_train_detect_balanced, y_train_detect_balanced = balance_data(X_train_detect, y_train_detect)

# Balance the class dataset
y_train_class_labels = np.argmax(y_train_class, axis=1)  # Convert one-hot to single label for balancing
X_train_class_balanced, y_train_class_balanced_labels = balance_data(X_train_class, y_train_class_labels)
y_train_class_balanced = pd.get_dummies(y_train_class_balanced_labels).values  # Convert back to one-hot


In [ ]:
class ELM(BaseEstimator, ClassifierMixin):
    def __init__(self, n_hidden_neurons=1000, activation_function='sigmoid'):
        self.n_hidden_neurons = n_hidden_neurons
        self.activation_function = activation_function
    
    def _activation(self, X):
        if self.activation_function == 'sigmoid':
            return 1 / (1 + np.exp(-X))
        elif self.activation_function == 'tanh':
            return np.tanh(X)
        else:
            raise ValueError('Unsupported activation function')
    
    def fit(self, X, y):
        # Initialize weights and biases
        self.input_weights = np.random.randn(X.shape[1], self.n_hidden_neurons) * 0.01
        self.biases = np.random.randn(self.n_hidden_neurons) * 0.01
        
        # Compute hidden layer output (H)
        H = self._activation(np.dot(X, self.input_weights) + self.biases[np.newaxis, :])
        
        # Compute output weights using the pseudo-inverse of H
        self.output_weights = np.dot(np.linalg.pinv(H), y)
    
    def predict(self, X):
        # Compute hidden layer output
        H = self._activation(np.dot(X, self.input_weights) + self.biases[np.newaxis, :])
        predictions = np.dot(H, self.output_weights)
        
        # Binary classification: Apply threshold
        return (predictions > 0.5).astype(int)

In [ ]:
# Initialize and train the ELM model for fault detection
elm_model_detect = ELM(n_hidden_neurons=500, activation_function='sigmoid')
print(elm_model_detect.fit(X_train_detect_balanced, y_train_detect_balanced))

# Predict on test data for fault detection
y_pred_detect = elm_model_detect.predict(X_test_detect)
y_pred_detect = np.round(y_pred_detect)


# Iterate through predictions and test data
for i, (true_val, pred_val) in enumerate(zip(y_test_detect, y_pred_detect)):
    if pred_val == 1:
        print(f"Fault detected in sample {i + 1}: Predicted value = {pred_val}, True value = {true_val}")
    else:
        print(f"No fault detected in sample {i + 1}: Predicted value = {pred_val}, True value = {true_val}")

# Evaluate the model for fault detection
accuracy_detect = accuracy_score(y_test_detect, y_pred_detect)
precision_detect = precision_score(y_test_detect, y_pred_detect, average='macro')
recall_detect = recall_score(y_test_detect, y_pred_detect, average='macro')
f1_detect = f1_score(y_test_detect, y_pred_detect, average='macro')
conf_matrix_detect = confusion_matrix(y_test_detect, y_pred_detect)

# Print evaluation metrics for fault detection
print('Fault Detection Evaluation Metrics:')
print('Accuracy:', accuracy_detect)
print('Precision:', precision_detect)
print('Recall:', recall_detect)
print('F1 Score:', f1_detect)
print('Confusion Matrix:\n', conf_matrix_detect)
print(f"Confusion Matrix:\n")
print(f"True No Fault (0), Predicted No Fault (0): {conf_matrix_detect[0, 0]}")
print(f"True No Fault (0), Predicted Fault (1): {conf_matrix_detect[0, 1]}")
print(f"True Fault (1), Predicted No Fault (0): {conf_matrix_detect[1, 0]}")
print(f"True Fault (1), Predicted Fault (1): {conf_matrix_detect[1, 1]}")
print(f"Number of actual faults in the test data: {sum(y_test_detect)}")


In [ ]:
# Initialize and train the ELM model for fault classification
elm_model_class = ELM(n_hidden_neurons=700, activation_function='sigmoid')
elm_model_class.fit(X_train_class_balanced, y_train_class_balanced)

# Predict on test data for fault classification
y_pred_class = elm_model_class.predict(X_test_class)
y_pred_class = np.round(y_pred_class)

# Evaluate the model for fault classification
accuracy_class = accuracy_score(y_test_class.argmax(axis=1), y_pred_class.argmax(axis=1))
precision_class = precision_score(y_test_class.argmax(axis=1), y_pred_class.argmax(axis=1), average='macro')
recall_class = recall_score(y_test_class.argmax(axis=1), y_pred_class.argmax(axis=1), average='macro')
f1_class = f1_score(y_test_class.argmax(axis=1), y_pred_class.argmax(axis=1), average='macro')
conf_matrix_class = confusion_matrix(y_test_class.argmax(axis=1), y_pred_class.argmax(axis=1))

# Print evaluation metrics for fault classification
print('Fault Classification Evaluation Metrics:')
print('Accuracy:', accuracy_class)
print('Precision:', precision_class)
print('Recall:', recall_class)
print('F1 Score:', f1_class)
print('Confusion Matrix:\n', conf_matrix_class)

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Plot confusion matrix for fault detection
plt.figure(figsize=(8, 6))
plot_confusion_matrix(conf_matrix_detect, classes=['No Fault', 'Fault'], title='Confusion Matrix for Fault Detection')
plt.show()

# Plot confusion matrix for fault classification (if it applies)
plt.figure(figsize=(8, 6))
plot_confusion_matrix(conf_matrix_class, classes=['Class 0', 'Class 1'], title='Confusion Matrix for Fault Classification')
plt.show()

# Create a figure for evaluation metrics for fault detection and classification
plt.figure(figsize=(12, 6))

# Plot metrics for fault detection
plt.subplot(1, 2, 1)
sns.barplot(x='Metric', y='Value', data=metrics_detect)
plt.title('Evaluation Metrics for Fault Detection')
plt.ylim(0, 1)
plt.xticks(rotation=45)

# Plot metrics for fault classification
plt.subplot(1, 2, 2)
sns.barplot(x='Metric', y='Value', data=metrics_class)
plt.title('Evaluation Metrics for Fault Classification')
plt.ylim(0, 1)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



# # Function to plot confusion matrix
# def plot_confusion_matrix(cm, title='Confusion Matrix', cmap=plt.cm.Blues):
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(np.unique(cm)))
#     plt.xticks(tick_marks, np.unique(cm), rotation=45)
#     plt.yticks(tick_marks, np.unique(cm))
#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

# # Plot confusion matrix for fault detection
# plt.figure(figsize=(8, 6))
# plot_confusion_matrix(conf_matrix_detect, title='Confusion Matrix for Fault Detection')
# plt.show()

# # Plot confusion matrix for fault classification
# plt.figure(figsize=(8, 6))
# plot_confusion_matrix(conf_matrix_class, title='Confusion Matrix for Fault Classification')
# plt.show()

# # Plot evaluation metrics
# metrics_detect = pd.DataFrame({
#     'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
#     'Value': [accuracy_detect, precision_detect, recall_detect, f1_detect]
# })

# metrics_class = pd.DataFrame({
#     'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
#     'Value': [accuracy_class, precision_class, recall_class, f1_class]
# })

# plt.figure(figsize=(12, 6))
# plt.subplot(1, 2, 1)
# sns.barplot(x='Metric', y='Value', data=metrics_detect)
# plt.title('Evaluation Metrics for Fault Detection')
# plt.ylim(0, 1)

# plt.subplot(1, 2, 2)
# sns.barplot(x='Metric', y='Value', data=metrics_class)
# plt.title('Evaluation Metrics for Fault Classification')
# plt.ylim(0, 1)

# plt.show()